In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from ConvAutoencoder import Encoder, Decoder
from actor_critic_rnn import ActorCriticRNN, update, compute_reward
import matplotlib.pyplot as plt
import torch.nn.functional as F
import mujoco
import glfw
import numpy as np
import time
from PIL import Image

In [2]:
XML_FILE_PATH = "/Users/edoardozappia/Desktop/Tesi_Magistrale/irregular_shape_2D.xml"

torch.autograd.set_detect_anomaly(True)

def save_image_to_array(rgb_buffer, width, height, target_size):
    """
    Salva un'immagine dalla simulazione in un array ridimensionato alla dimensione target_size.
    """
    # Converti il buffer in un'immagine
    img = np.frombuffer(rgb_buffer, dtype=np.uint8).reshape(height, width, 3)
    img = Image.fromarray(img)

    # Ridimensiona l'immagine
    img_resized = img.resize(target_size, Image.LANCZOS)

    # Converti l'immagine ridimensionata in un array numpy
    return np.array(img_resized)

def visualize_frames(real_frame, predicted_frame, frame_count):
    """
    Visualizza i frame reali e predetti.
    """
    plt.figure(figsize=(10, 5))

    # Frame reale
    plt.subplot(1, 2, 1)
    plt.imshow(real_frame)
    plt.title(f"Real Frame - {frame_count}")
    plt.axis('off')

    # Frame predetto
    plt.subplot(1, 2, 2)
    plt.imshow(predicted_frame)
    plt.title(f"Predicted Frame - {frame_count}")
    plt.axis('off')

    plt.show()

def main():
    # Parametri del movimento sinusoidale
    freq_x = 0.5  # Frequenza della sinusoide lungo x
    freq_y = 0.3  # Frequenza della sinusoide lungo y
    freq_phi = 0.1  # Frequenza della sinusoide per la rotazione
    amp_x = 5000.0  # Ampiezza della sinusoide lungo x
    amp_y = 5000.0  # Ampiezza della sinusoide lungo y
    amp_phi = 1.0  # Ampiezza della sinusoide per la rotazione
    R = 0.2  # Raggio massimo della circonferenza
    dt = 0.01  # Timestep fisso
    noise = 0.0  # Rumore bianco gaussiano

    # Parametri del modello
    latent_dim = 8192  # Modifica in base al tuo modello
    action_dim = 8192  # Modifica in base al tuo modello

    # Inizializza il modello
    actor_critic_rnn = ActorCriticRNN(latent_dim, action_dim)

    # Encoder pre-addestrato
    encoder = Encoder()
    encoder.load_state_dict(torch.load('encoder.pth'))
    encoder.eval()

    # Decoder pre-addestrato
    decoder = Decoder()
    decoder.load_state_dict(torch.load('decoder.pth'))
    decoder.eval()  

    if not glfw.init():
        print("Impossibile inizializzare GLFW")
        return

    glfw.window_hint(glfw.VISIBLE, glfw.TRUE)
    window = glfw.create_window(800, 800, "MuJoCo Viewer", None, None)
    if not window:
        print("Errore durante la creazione della finestra GLFW")
        glfw.terminate()
        return

    glfw.make_context_current(window)

    model = mujoco.MjModel.from_xml_path(XML_FILE_PATH)
    data = mujoco.MjData(model)

    # Configura la telecamera
    camera = mujoco.MjvCamera()
    camera.type = mujoco.mjtCamera.mjCAMERA_FREE
    camera.lookat = np.array([0, 0, 0])
    camera.distance = 2.0
    camera.azimuth = 90
    camera.elevation = -90

    # Renderer per visualizzare la finestra
    scene = mujoco.MjvScene(model, maxgeom=1000)
    context = mujoco.MjrContext(model, mujoco.mjtFontScale.mjFONTSCALE_150)

    t = 0  # Tempo iniziale

    frame_count = 0

    # Ottieni dimensioni del viewport
    viewport_width, viewport_height = glfw.get_framebuffer_size(window)

    # Prealloca buffer per RGB e profondità
    rgb_buffer = np.zeros((viewport_height, viewport_width, 3), dtype=np.uint8)
    depth_buffer = np.zeros((viewport_height, viewport_width), dtype=np.float32)

    k = 0
    max_steps = 100000
    max_episodes = 5000
    episode_termination_threshold = 10.0
    counter_threshold_decr = 500

    optimizer = optim.Adam(actor_critic_rnn.parameters(), lr=1e-3)

    #while not glfw.window_should_close(window):
    
    fixed_movement = True # per avere stesso movimento all'inizio di ogni episodio

    for episode in range(max_episodes):
     if fixed_movement:
        t = 0  # Tempo iniziale fisso per movimento ripetibile
     else:
        t = np.random.uniform(0, 2 * np.pi)  # Tempo iniziale casuale per variazioni

     if counter_threshold_decr == episode:
        episode_termination_threshold -= 0.04
        counter_threshold_decr += 250

     mujoco.mj_resetData(model, data)

     viewport = mujoco.MjrRect(0, 0, viewport_width, viewport_height)
     mujoco.mjr_readPixels(rgb_buffer, depth_buffer, viewport, context)
     frame_array = save_image_to_array(rgb_buffer.tobytes(), viewport_width, viewport_height, target_size=(64, 64))

     frame = torch.tensor(frame_array).float() / 255.0
     frame = frame.permute(2, 0, 1)
     frame = frame.unsqueeze(0)
     with torch.no_grad():
        latent_state = encoder(frame)
        latent_state = latent_state.view(latent_state.size(0), -1).unsqueeze(1)

     episode_reward = 0.0
     for step in range(max_steps):
        
        # Aggiorna il tempo
        t += dt

        # Forze sinusoidali
        force_x = amp_x * np.sin(2 * np.pi * freq_x * t) + noise * np.random.randn()
        force_y = amp_y * np.cos(2 * np.pi * freq_y * t) + noise * np.random.randn()

        # Momento sinusoidale
        torque_phi = amp_phi * np.sin(2 * np.pi * freq_phi * t) + noise * np.random.randn()

        # Controlla che l'oggetto rimanga dentro la circonferenza
        distance = np.sqrt(data.qpos[0] ** 2 + data.qpos[1] ** 2)
        if distance >= R:
            # Forza di rimbalzo verso il centro
            direction_x = -data.qpos[0] / distance
            direction_y = -data.qpos[1] / distance
            force_x += direction_x * 1000.0
            force_y += direction_y * 1000.0

        # Applica le forze e i momenti
        data.qfrc_applied[0] = force_x
        data.qfrc_applied[1] = force_y
        data.qfrc_applied[2] = torque_phi

        # Avanza la simulazione
        mujoco.mj_step(model, data)

        # Renderizza la scena
        mujoco.mjv_updateScene(model, data, mujoco.MjvOption(), None, camera, mujoco.mjtCatBit.mjCAT_ALL, scene)
        viewport = mujoco.MjrRect(0, 0, viewport_width, viewport_height)
        mujoco.mjr_render(viewport, scene, context)

        # Leggi i pixel RGB
        mujoco.mjr_readPixels(rgb_buffer, depth_buffer, viewport, context)

        # Salva il frame corrente come array
        frame_array = save_image_to_array(rgb_buffer.tobytes(), viewport_width, viewport_height, target_size=(64, 64))

        frame = torch.tensor(frame_array).float() / 255.0
        frame = frame.permute(2, 0, 1)
        frame = frame.unsqueeze(0)

        # Ottieni lo stato latente
        with torch.no_grad():
            real_next_latent_frame = encoder(frame)
            real_next_latent_state = real_next_latent_frame.view(real_next_latent_frame.size(0), -1).unsqueeze(1)

        action, _ = actor_critic_rnn(latent_state.clone().detach())

        predicted_next_state = latent_state.clone().detach() + action

        # Termina l'episodio se lo stato predetto diverge troppo dal reale
        if torch.norm(predicted_next_state.detach() - real_next_latent_state.detach()).item() > episode_termination_threshold:
            print(f"Episode {episode} terminated early at step {step}.")
            break

        # Aggiorna il modello
        loss, actor_loss, critic_loss, reward = update(
            actor_critic_rnn, optimizer, latent_state, real_next_latent_state, gamma=0.99
        )

        episode_reward += reward

        print(f"Step {step}, Reward: {reward:.4f}")

        # Imposta lo stato successivo
        latent_state = predicted_next_state.clone().detach()
        
        glfw.swap_buffers(window)
        glfw.poll_events()
        time.sleep(dt)
     
     norm_coeff = step if step > 0 else 1

     print(f"Episode {episode + 1}/{max_episodes}, Mean Reward: {episode_reward/step:.4f}")

    glfw.destroy_window(window)
    glfw.terminate()

    # Salva i pesi del modello Actor-Critic RNN
    torch.save(actor_critic_rnn.state_dict(), "actor_critic_rnn.pth")

    print("Modello Actor-Critic RNN salvato correttamente.")


if __name__ == "__main__":
    main()


Episode 0 terminated early at step 0.
Episode 1/5000, Total Reward: 0.0000
Episode 1 terminated early at step 0.
Episode 2/5000, Total Reward: 0.0000
Episode 2 terminated early at step 0.
Episode 3/5000, Total Reward: 0.0000
Episode 3 terminated early at step 0.
Episode 4/5000, Total Reward: 0.0000
Episode 4 terminated early at step 0.
Episode 5/5000, Total Reward: 0.0000
Episode 5 terminated early at step 0.
Episode 6/5000, Total Reward: 0.0000
Episode 6 terminated early at step 0.
Episode 7/5000, Total Reward: 0.0000
Episode 7 terminated early at step 0.
Episode 8/5000, Total Reward: 0.0000
Episode 8 terminated early at step 0.
Episode 9/5000, Total Reward: 0.0000
Episode 9 terminated early at step 0.
Episode 10/5000, Total Reward: 0.0000
Episode 10 terminated early at step 0.
Episode 11/5000, Total Reward: 0.0000
Episode 11 terminated early at step 0.
Episode 12/5000, Total Reward: 0.0000
Episode 12 terminated early at step 0.
Episode 13/5000, Total Reward: 0.0000
Episode 13 termina

KeyboardInterrupt: 

In [6]:
XML_FILE_PATH = "/Users/edoardozappia/Desktop/Tesi_Magistrale/irregular_shape_2D.xml"

def save_image_to_array(rgb_buffer, width, height, target_size):
    """
    Salva un'immagine dalla simulazione in un array ridimensionato alla dimensione target_size.
    """
    # Converti il buffer in un'immagine
    img = np.frombuffer(rgb_buffer, dtype=np.uint8).reshape(height, width, 3)
    img = Image.fromarray(img)

    # Ridimensiona l'immagine
    img_resized = img.resize(target_size, Image.LANCZOS)

    # Converti l'immagine ridimensionata in un array numpy
    return np.array(img_resized)

def visualize_frames(real_frame, predicted_frame, frame_count):
    """
    Visualizza i frame reali e predetti.
    """
    plt.figure(figsize=(10, 5))

    # Frame reale
    plt.subplot(1, 2, 1)
    plt.imshow(real_frame)
    plt.title(f"Real Frame - {frame_count}")
    plt.axis('off')

    # Frame predetto
    plt.subplot(1, 2, 2)
    plt.imshow(predicted_frame)
    plt.title(f"Predicted Frame - {frame_count}")
    plt.axis('off')

    plt.show()

def main():
    # Parametri del movimento sinusoidale
    freq_x = 0.5  # Frequenza della sinusoide lungo x
    freq_y = 0.3  # Frequenza della sinusoide lungo y
    freq_phi = 0.1  # Frequenza della sinusoide per la rotazione
    amp_x = 5000.0  # Ampiezza della sinusoide lungo x
    amp_y = 5000.0  # Ampiezza della sinusoide lungo y
    amp_phi = 1.0  # Ampiezza della sinusoide per la rotazione
    R = 0.2  # Raggio massimo della circonferenza
    dt = 0.01  # Timestep fisso
    noise = 0.0  # Rumore bianco gaussiano

    # Parametri del modello
    latent_dim = 8192  # Modifica in base al tuo modello
    action_dim = 8192  # Modifica in base al tuo modello

    # Inizializza il modello
    actor_critic_rnn = ActorCriticRNN(latent_dim, action_dim)

    # Carica i pesi salvati
    actor_critic_rnn.load_state_dict(torch.load("actor_critic_rnn.pth"))
    actor_critic_rnn.eval()  # Imposta il modello in modalità valutazione

    # Encoder pre-addestrato
    encoder = Encoder()
    encoder.load_state_dict(torch.load('encoder.pth'))
    encoder.eval()

    # Decoder pre-addestrato
    decoder = Decoder()
    decoder.load_state_dict(torch.load('decoder.pth'))
    decoder.eval()  

    if not glfw.init():
        print("Impossibile inizializzare GLFW")
        return

    glfw.window_hint(glfw.VISIBLE, glfw.TRUE)
    window = glfw.create_window(800, 800, "MuJoCo Viewer", None, None)
    if not window:
        print("Errore durante la creazione della finestra GLFW")
        glfw.terminate()
        return

    glfw.make_context_current(window)

    model = mujoco.MjModel.from_xml_path(XML_FILE_PATH)
    data = mujoco.MjData(model)

    # Configura la telecamera
    camera = mujoco.MjvCamera()
    camera.type = mujoco.mjtCamera.mjCAMERA_FREE
    camera.lookat = np.array([0, 0, 0])
    camera.distance = 2.0
    camera.azimuth = 90
    camera.elevation = -90

    # Renderer per visualizzare la finestra
    scene = mujoco.MjvScene(model, maxgeom=1000)
    context = mujoco.MjrContext(model, mujoco.mjtFontScale.mjFONTSCALE_150)

    t = 0  # Tempo iniziale

    frame_count = 0

    # Ottieni dimensioni del viewport
    viewport_width, viewport_height = glfw.get_framebuffer_size(window)

    # Prealloca buffer per RGB e profondità
    rgb_buffer = np.zeros((viewport_height, viewport_width, 3), dtype=np.uint8)
    depth_buffer = np.zeros((viewport_height, viewport_width), dtype=np.float32)


    k = 0




    while not glfw.window_should_close(window):
        
        # Aggiorna il tempo
        t += dt

        # Forze sinusoidali
        force_x = amp_x * np.sin(2 * np.pi * freq_x * t) + noise * np.random.randn()
        force_y = amp_y * np.cos(2 * np.pi * freq_y * t) + noise * np.random.randn()

        # Momento sinusoidale
        torque_phi = amp_phi * np.sin(2 * np.pi * freq_phi * t) + noise * np.random.randn()

        # Controlla che l'oggetto rimanga dentro la circonferenza
        distance = np.sqrt(data.qpos[0] ** 2 + data.qpos[1] ** 2)
        if distance >= R:
            # Forza di rimbalzo verso il centro
            direction_x = -data.qpos[0] / distance
            direction_y = -data.qpos[1] / distance
            force_x += direction_x * 1000.0
            force_y += direction_y * 1000.0

        # Applica le forze e i momenti
        data.qfrc_applied[0] = force_x
        data.qfrc_applied[1] = force_y
        data.qfrc_applied[2] = torque_phi

        # Avanza la simulazione
        mujoco.mj_step(model, data)

        # Debug: verifica posizione e velocità
        #print(f"Tempo: {t:.2f}, Posizione: x={data.qpos[0]:.3f}, y={data.qpos[1]:.3f}, phi={data.qpos[2]:.3f}")
        #print(f"Velocità: x_vel={data.qvel[0]:.3f}, y_vel={data.qvel[1]:.3f}, phi_vel={data.qvel[2]:.3f}")

        # Renderizza la scena
        mujoco.mjv_updateScene(model, data, mujoco.MjvOption(), None, camera, mujoco.mjtCatBit.mjCAT_ALL, scene)
        viewport = mujoco.MjrRect(0, 0, viewport_width, viewport_height)
        mujoco.mjr_render(viewport, scene, context)

        # Leggi i pixel RGB
        mujoco.mjr_readPixels(rgb_buffer, depth_buffer, viewport, context)

        # Salva il frame corrente come array
        frame_array = save_image_to_array(rgb_buffer.tobytes(), viewport_width, viewport_height, target_size=(64, 64))

        frame = torch.tensor(frame_array).float() / 255.0
        frame = frame.permute(2, 0, 1)
        frame = frame.unsqueeze(0)

        # Ottieni lo stato latente
        with torch.no_grad():
            latent_state = encoder(frame)
            latent_state = latent_state.view(latent_state.size(0), -1).unsqueeze(1)
        
        # Decodifica il frame reale e predetto
        real_frame_decoded = torch.tensor(latent_state).float()
        real_frame_decoded = real_frame_decoded.view(1, 128, 8, 8)
        real_frame_decoded = decoder(real_frame_decoded).detach().squeeze(0).permute(1, 2, 0).cpu().numpy()
        
        if frame_count != 0:
            # Visualizza i frame reali e predetti
            #visualize_frames(real_frame_decoded, predicted_frame_decoded, frame_count)
            print(f"Differenza stato predetto e stato reale: {torch.norm(latent_state - predicted_state)}")

        if frame_count == 0:
            action_0, _ = actor_critic_rnn(latent_state)

        action, _ = actor_critic_rnn(latent_state)

        if torch.norm(action_0 - action) == 0:
            k += 1
        
        print(f"Azioni uguali: {k}")
        predicted_state = latent_state + action

        print(f"Norma azione: {torch.norm(action)}")

        action_image = torch.tensor(action).float()
        action_image = action_image.view(1, 128, 8, 8)
        action_image = decoder(action_image).detach().squeeze(0).permute(1, 2, 0).cpu().numpy()

        #visualize_frames(action_image, real_frame_decoded, frame_count)

        predicted_frame_decoded = torch.tensor(predicted_state).float()
        predicted_frame_decoded = predicted_frame_decoded.view(1, 128, 8, 8)
        predicted_frame_decoded = decoder(predicted_frame_decoded).detach().squeeze(0).permute(1, 2, 0).cpu().numpy()

        frame_count += 1

        glfw.swap_buffers(window)
        glfw.poll_events()
        time.sleep(dt)

    glfw.destroy_window(window)
    glfw.terminate()


if __name__ == "__main__":
    main()


/var/folders/72/24j7jy9s6sb2xm_dc84kdhm00000gn/T/ipykernel_48134/2926913652.py:169: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  real_frame_decoded = torch.tensor(latent_state).float()
/var/folders/72/24j7jy9s6sb2xm_dc84kdhm00000gn/T/ipykernel_48134/2926913652.py:191: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  action_image = torch.tensor(action).float()
/var/folders/72/24j7jy9s6sb2xm_dc84kdhm00000gn/T/ipykernel_48134/2926913652.py:197: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_frame_decoded = torch.tensor(predicted_state

Azioni uguali: 1
Norma azione: 9.314242362976074
Differenza stato predetto e stato reale: 9.357549667358398
Azioni uguali: 1
Norma azione: 9.29541015625
Differenza stato predetto e stato reale: 9.32781982421875
Azioni uguali: 1
Norma azione: 9.237226486206055
Differenza stato predetto e stato reale: 9.253464698791504
Azioni uguali: 1
Norma azione: 9.208675384521484
Differenza stato predetto e stato reale: 9.2437105178833
Azioni uguali: 1
Norma azione: 9.178071975708008
Differenza stato predetto e stato reale: 9.222015380859375
Azioni uguali: 1
Norma azione: 9.144834518432617
Differenza stato predetto e stato reale: 9.185683250427246
Azioni uguali: 1
Norma azione: 9.099295616149902
Differenza stato predetto e stato reale: 9.117992401123047
Azioni uguali: 1
Norma azione: 9.112372398376465
Differenza stato predetto e stato reale: 9.144478797912598
Azioni uguali: 1
Norma azione: 9.15002155303955
Differenza stato predetto e stato reale: 9.180731773376465
Azioni uguali: 1
Norma azione: 9.142